NeuralChat is a customizable chat framework designed to create user own chatbot within few minutes on multiple architectures. This notebook is used to demonstrate how to build a talking chatbot on 4th Generation of Intel® Xeon® Scalable Processors Sapphire Rapids.

The 4th Generation of Intel® Xeon® Scalable processor provides two instruction sets viz. AMX_BF16 and AMX_INT8 which provides acceleration for bfloat16 and int8 operations respectively.

# Prepare Environment

Install intel extension for transformers:

In [ ]:
!pip install intel-extension-for-transformers

Install Requirements:

In [ ]:
!git clone https://github.com/intel/intel-extension-for-transformers.git

In [ ]:
%cd ./intel-extension-for-transformers/intel_extension_for_transformers/neural_chat/
!pip install -r requirements_cpu.txt
%cd ../../../

# Build your chatbot 💻

## Text Chat

Giving NeuralChat the textual instruction, it will respond with the textual response.

In [1]:
# BF16 Optimization
from intel_extension_for_transformers.neural_chat import build_chatbot, PipelineConfig
from intel_extension_for_transformers.transformers import MixedPrecisionConfig
config = PipelineConfig(optimization_config=MixedPrecisionConfig())
chatbot = build_chatbot(config)
response = chatbot.predict(query="Tell  me about Lionel Messi")
print(response)


/home/u51a924cab5fb25f87f4f9c01e5e094f/.conda/envs/itrex-1/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/home/u51a924cab5fb25f87f4f9c01e5e094f/.conda/envs/itrex-1/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading model Intel/neural-chat-7b-v3-1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Lionel Andrés Messi is an Argentine professional footballer widely regarded as one of the greatest players in the history of the sport. Born on June 24, 1987, in Rosario, Argentina, he began his career with Newell's Old Boys before joining FC Barcelona at the age of 13.

Messi has won numerous individual awards throughout his career, including six Ballon d'Or titles, which is a record shared with Cristiano Ronaldo. He holds the record for most goals scored in La Liga (474), the UEFA Champions League (80), and the Copa América (6). As of now, he has scored over 750 official senior career goals across all competitions.

Apart from his impressive goal-scoring abilities, Messi is known for his exceptional dribbling skills, vision, and creativity on the field. His ability to read the game and anticipate opponents' movements have made him a key player for both club and country.

In addition to his successes with FC Barcelona, where he has won ten La Liga titles, four UEFA Champions League tr

## Text Chat With Retrieval Plugin

User could also leverage NeuralChat Retrieval plugin to do domain specific chat by feding with some documents like below:

In [ ]:
%cd ./intel-extension-for-transformers/intel_extension_for_transformers/neural_chat/pipeline/plugins/retrieval/
!pip install -r requirements.txt
%cd ../../../../../../

In [ ]:
!mkdir docs
%cd docs
!curl -OL https://raw.githubusercontent.com/intel/intel-extension-for-transformers/main/intel_extension_for_transformers/neural_chat/assets/docs/sample.jsonl
!curl -OL https://raw.githubusercontent.com/intel/intel-extension-for-transformers/main/intel_extension_for_transformers/neural_chat/assets/docs/sample.txt
!curl -OL https://raw.githubusercontent.com/intel/intel-extension-for-transformers/main/intel_extension_for_transformers/neural_chat/assets/docs/sample.xlsx
%cd ..

In [ ]:
from intel_extension_for_transformers.neural_chat import PipelineConfig
from intel_extension_for_transformers.neural_chat import build_chatbot
from intel_extension_for_transformers.neural_chat import plugins
plugins.retrieval.enable=True
plugins.retrieval.args["input_path"]="./docs/"
config = PipelineConfig(plugins=plugins)
chatbot = build_chatbot(config)
response = chatbot.predict("How many cores does the Intel® Xeon® Platinum 8480+ Processor have in total?")
print(response)

## Voice Chat with ASR & TTS Plugin

In the context of voice chat, users have the option to engage in various modes: utilizing input audio and receiving output audio, employing input audio and receiving textual output, or providing input in textual form and receiving audio output.

For the Python API code, users have the option to enable different voice chat modes by setting ASR and TTS plugins enable or disable.

In [ ]:
%cd ./intel-extension-for-transformers/intel_extension_for_transformers/neural_chat/pipeline/plugins/audio/
!pip install -r requirements.txt
%cd ../../../../../../

In [ ]:
!curl -OL https://raw.githubusercontent.com/intel/intel-extension-for-transformers/main/intel_extension_for_transformers/neural_chat/assets/speaker_embeddings/spk_embed_default.pt
!curl -OL https://raw.githubusercontent.com/intel/intel-extension-for-transformers/main/intel_extension_for_transformers/neural_chat/assets/audio/sample.wav

In [3]:
from intel_extension_for_transformers.neural_chat import PipelineConfig
from intel_extension_for_transformers.neural_chat import build_chatbot
from intel_extension_for_transformers.neural_chat import plugins
plugins.tts.enable = True
plugins.tts.args["output_audio_path"] = "./response.wav"
plugins.asr.enable = True

config = PipelineConfig(plugins=plugins)
chatbot = build_chatbot(config)
result = chatbot.predict(query="./sample")
print(result)

/home/u51a924cab5fb25f87f4f9c01e5e094f/.conda/envs/itrex-1/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


create tts plugin instance...
plugin parameters:  {'output_audio_path': './response.wav'}
create asr plugin instance...
plugin parameters:  {}


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading model Intel/neural-chat-7b-v3-1


/home/u51a924cab5fb25f87f4f9c01e5e094f/.conda/envs/itrex-1/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2024-07-12 10:05:33,732 - tts.py - root - WARNING - No customized speaker embedding is found! Use the default one


./response.wav


# Low Precision Optimization

## BF16

In [2]:
# BF16 Optimization
from intel_extension_for_transformers.neural_chat.config import PipelineConfig
from intel_extension_for_transformers.transformers import MixedPrecisionConfig
config = PipelineConfig(optimization_config=MixedPrecisionConfig())
chatbot = build_chatbot(config)
response = chatbot.predict(query="Tell me about Mar Baselios College of Engineering and Technology (Autonomous).")
print(response)

Loading model Intel/neural-chat-7b-v3-1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Mar Baselios College of Engineering and Technology (Autonomous) is a prestigious institution located in Thiruvananthapuram, Kerala, India. Established with the aim of providing quality technical education, it has grown into a renowned autonomous college under the aegis of Mar Baselios Educational Society. The college offers various undergraduate and postgraduate programs in engineering disciplines such as Computer Science & Engineering, Electronics & Communication Engineering, Mechanical Engineering, Civil Engineering, and Information Technology.

The autonomous status granted to this college allows it to have greater flexibility in designing its curriculum, conducting research projects, and implementing innovative teaching methodologies. This autonomy enables the faculty members to stay updated with the latest advancements in their respective fields and incorporate them into the learning process.

Mar Baselios College of Engineering and Technology (Autonomous) is committed to fosterin